# цели и задачи проекта;
Цель проекта: увеличить эффективность работы менеджеров, за счет нового подхода к отбору объектов недвижимости для анализа, что увеличит рабочее время на более выгодные предложения на рынке

Задачи проекта:
1) Создать модели классификации и ранжирования недвижимости;
2) Должно быть разработано несколько моделей в зависимости от цели поиска недвижимости;

# поэтапное описание работы;
1) Анализ датасета;
2) Разработка вариантов преобразования датасета;
3) Проверка преобразования на полноту и точность;
4) Изучение общепринятых критериев оценки недвижимости;
5) Построение подели на выявленных критериях и попытки формулировки цели поиска недвижимости;
6) Проверка модели;
7) Проба вариантов изменения критериев;
8) Принятие решение о итоговой моделях;

# промежуточные выводы;
1) В Датасете надо чистить все поля;
2) Поле "homeFacts" не является валидным JSON;
3) Попытки бороться с полем цены:
    '+'
    '\m' 
4) Площади недвижимости мало значений - колонку не учитываем;


# основные итоги работы.


# Предполагается наличие файла data 2.csv в папке с проектом

In [267]:
#иморты
import pandas as pd
import json
import re

In [303]:
#функции

#преобразование цены в число, убираем $ и запятые
def convTarget(s):
    if isinstance(s, str) and s != '':
        s = s.replace(',','')
        if len(s)>4 and s[-4] == 'sqft':
            s = s[:-4]
        if s[0] == '$':
            s = s[1:]
        if len(s)>2 and s[-2:] == '/m':
            s = s[:-2]
        if len(s)>3 and s[-3:] == '/mo':
            s = s[:-3]
        if s[-1] == '+':
            s = s[:-1]
        try:
            s = float(s)
        except BaseException:
            s = 0
        return s

#преобразование HomeFacts
def convHomeFacts(s):
    s = s.replace('None', 'null')
    s = s.replace("'", '"')
    try:
        s = json.loads(s)
    except BaseException:
        s = 'NONE'
    return s

#парсинг словаря для заполнения новых колонок
def parseHomeFacts(tmp_dict, name):
    for i in range (len(tmp_dict['atAGlanceFacts'])):
        if tmp_dict['atAGlanceFacts'][i]['factLabel'] == name:
            return tmp_dict['atAGlanceFacts'][i]['factValue']

#парсинг текстовых значений в бинарное
def convYesNo(s):
    if not s:
        return True

#преобразование площади
def convSqFt(s):   
    s = s.replace('nan','').strip()
    s = s.replace('nan nan','').strip()
    s = s.replace('0.0 ','').strip()
    s = s.replace(' 0.0','').strip()
    s = s.strip()
    if s.find(' ') > 0:
        try:
            s1 = s
            tmp = s.split(' ')
            s = float(tmp[0]) + float(tmp[1])
        except BaseException:
            print('ERROR_split_' + s1 + '_')
    try:
        float(s)
    except BaseException:
        s = 0
    return s

#преобразуем статус - убираем редкие статусы
def convStatus(s):
    try:
        s = s.lower()
    except BaseException:
        s = ''
    if not s in ['for sale', 'active', 'foreclosure', 'new construction', 'pending', 'pre-foreclosure']:
        s = None
    return s

In [310]:
#загрузка датасета
df = pd.read_csv('data 2.csv', encoding='UTF-8')
start_size = df.shape[0]
#первичная обработка
df = df.dropna(axis=0, subset=['sqft'])
df['target'] = df['target'].fillna(0)
df['fireplace'] = df['fireplace'].fillna(False)
df['target'] = df['target'].apply(lambda x: convTarget(x)) #цена в число
df = df[(df['target'] != 0)]

#преобразуем и чистим не валидные JSON в поле homeFacts
df = df[df['homeFacts'].apply(lambda x: x.count("'")).isin([56,58])]
df['homeFacts'] = df['homeFacts'].apply(lambda x: convHomeFacts(x))
df = df[df['homeFacts'] != 'NONE']

#добавляем новые колонки на основе homeFacts
new_lables = []
tmp_dict = df['homeFacts'][0] 
for i in range (len(tmp_dict['atAGlanceFacts'])):
    new_lables.append(tmp_dict['atAGlanceFacts'][i]['factLabel']) #названия колонок
for item in new_lables:
    df[item] = df['homeFacts'].apply(lambda x: parseHomeFacts(x,item)) #добавляем в датасет
df = df.drop(columns=['homeFacts'])

#преобразуем sqft - обединяем с lotsize
# df['sqft'] = df['sqft'].apply(lambda x: convTarget(x))
# df['lotsize'] = df['lotsize'].apply(lambda x: convTarget(x))
# df['SqFt'] = (df['sqft'].astype(str) +' '+ df['lotsize'].astype(str))
# df['SqFt'] = df['SqFt'].apply(lambda x: convSqFt(x))
df = df.drop(columns=['sqft','lotsize']) #не проверил - значений мало для анализа

#fireplace к единому виду
df['fireplace'] = df['fireplace'].apply(lambda x: convYesNo(x))

#PrivatePool к единому виду - собираем из двух - дропаем исходные
df['Private Pool'] = df['PrivatePool'].astype(str) + df['private pool'].astype(str)
df['Private Pool'] = df['Private Pool'].apply(lambda x: None if x == 'nannan' else 'yes')
df = df.drop(columns=['PrivatePool','private pool'])

#колонка status
df['status'] = df['status'].astype(str).apply(lambda x: convStatus(x))
df = df.dropna(axis=0, subset=['status'])

current_size=df.shape[0]
loss_size= start_size - current_size


In [316]:
df['propertyType'].value_counts().index.tolist()

['single-family home',
 'Single Family',
 'Single Family Home',
 'condo',
 'townhouse',
 'Land',
 'multi-family',
 'Condo/Townhome/Row Home/Co-Op',
 'Traditional',
 'Condo',
 'lot/land',
 'Townhouse',
 'coop',
 'Ranch',
 'High Rise',
 'Single Detached, Traditional',
 'Contemporary',
 'mobile/manufactured',
 'Detached, One Story',
 'Multi Family',
 'Multi-Family Home',
 'Colonial',
 'Contemporary/Modern',
 '1 Story',
 'Single Detached',
 'Mfd/Mobile Home',
 '2 Stories',
 'Detached, Two Story',
 'Mobile / Manufactured',
 'Transitional',
 'Florida',
 'One Story',
 'Apartment',
 'Bungalow',
 'Farms/Ranches',
 'apartment',
 'Garden Home',
 'Two Story',
 'Spanish/Mediterranean',
 'Cape Cod',
 'Other Style',
 'Condo/Townhome, Traditional',
 '2 Story',
 'Craftsman',
 'Contemporary/Modern, Traditional',
 '2 Stories, Traditional',
 'Cooperative',
 'Condo/Townhome',
 'Condo/Townhome, Contemporary/Modern',
 'SingleFamilyResidence',
 'Other',
 'Single Detached, Contemporary/Modern',
 '1 Story, Trad

In [ ]:
def convPropertyType(s):
    s = s.lower()
    if s.find('single') > 0
        s = 'Single Family'
    if s.find('multi') > 0
        s = 'Multi Family'

In [311]:
#Какие-то итоговые показатели
#Показатели для модели:
#'city', 
    

print('Итоговые показатели')
print('Потеряно записей',loss_size)

Итоговые показатели
Потеряно не валидных записейЖ 93376
